# Preprocessing - Association Rule Mining for Cardiovascular Comorbidity and Risk Factor Assessment

In data preprocessing, perform discretization and one-hot encoding.

1. Handling Missing Values: Missing or incomplete records are either imputed using appropriate statistical methods (e.g., mean, median, or mode) or removed if necessary, to prevent bias in the analysis.

2. Discretization (Binning): Continuous variables such as BMI, PhysicalHealth, MentalHealth, and SleepTime are transformed into categorical bins. This step converts numerical ranges into discrete categories (e.g., low, medium, high), which is required for association rule mining algorithms like Apriori that work on categorical data.

3. One-Hot Encoding: Categorical features are converted into binary vectors using one-hot encoding. This ensures that each category is represented as a separate column with values of 0 or 1, making the data suitable for mining frequent itemsets.

4. Final Dataset Verification: After preprocessing, the dataset is checked to confirm that all variables are properly discretized or encoded, and that it contains no missing or inconsistent values. This ensures reliable and interpretable association rules.

### Architecture Design
| Step | Action | Description |
| :---: | :--- | :--- |
| 1 | Input Data | Raw patient records from the CDC BRFSS heart disease dataset (Kaggle). |
| 2 | Preprocessing | Discretization and one-hot encoding (e.g., AgeCategory, BMI into Obese/Non-Obese, SleepTime into Short/Normal/Long) |
| 3 | Transactional Data | Output: Sparse DataFrame (each column is a specific medical feature/risk level). |
| 4 | Apriori Algorithm | Find Frequent Itemsets using a tuned minimum Support threshold. |
| 5 | Rule Generation | Generate rules (A $\implies$ B) based on a minimum Confidence threshold. |
| 6 | Evaluation & Filtering | Calculate Lift, Conviction, and Leverage. Filter for rules where Lift $>$ 1.0 (positive correlation). |
| 7 | Final Output | Ranked list of comorbidity rules, network graph visualization, item frequency bar charts, transaction length distribution, and co-occurrence heatmap |

In [46]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

### Load dataset

In [47]:
# Load dataset from Kaggle

df = pd.read_csv(r'C:\Users\Admin\Documents\dataset_association_mining\heart_2020_cleaned.csv')

print(df.head())
print(df.shape)

  HeartDisease    BMI Smoking  ... Asthma KidneyDisease  SkinCancer
0           No  16.60     Yes  ...    Yes            No         Yes
1           No  20.34      No  ...     No            No          No
2           No  26.58     Yes  ...    Yes            No          No
3           No  24.21      No  ...     No            No         Yes
4           No  23.71      No  ...     No            No          No

[5 rows x 18 columns]
(319795, 18)


### Check variable description of Numerical Columns

In [48]:
print(df[['BMI','PhysicalHealth','MentalHealth','SleepTime']].describe())


                 BMI  PhysicalHealth   MentalHealth      SleepTime
count  319795.000000    319795.00000  319795.000000  319795.000000
mean       28.325399         3.37171       3.898366       7.097075
std         6.356100         7.95085       7.955235       1.436007
min        12.020000         0.00000       0.000000       1.000000
25%        24.030000         0.00000       0.000000       6.000000
50%        27.340000         0.00000       0.000000       7.000000
75%        31.420000         2.00000       3.000000       8.000000
max        94.850000        30.00000      30.000000      24.000000


In [49]:
# Handle missing values

print(df.isna().sum())  # shows how many NaNs in each column
df = df.dropna() # drop rows with NaN values

HeartDisease        0
BMI                 0
Smoking             0
AlcoholDrinking     0
Stroke              0
PhysicalHealth      0
MentalHealth        0
DiffWalking         0
Sex                 0
AgeCategory         0
Race                0
Diabetic            0
PhysicalActivity    0
GenHealth           0
SleepTime           0
Asthma              0
KidneyDisease       0
SkinCancer          0
dtype: int64


In [50]:
# Discretize numeric attributes (e.g., BMI, PhysicalHealth, MentalHealth, SleepTime) using pd.cut()
# optional: classify types of obese based on bmi value
df['BMI_cat'] = pd.cut(df['BMI'], bins=[0, 18.5, 25, 30, 100], labels=['Underweight', 'Normal', 'Overweight', 'Obese'])

# values represents no physical unhealthy days
# include 0 in 'None' category
df['PhysicalHealth_cat'] = pd.cut(df['PhysicalHealth'], bins=[-1, 0, 3, 7, 30], labels=['None', 'Low', 'Moderate', 'High'])

# values represents no mental unhealthy days
# include 0 in 'None' category
df['MentalHealth_cat'] = pd.cut(df['MentalHealth'], bins=[-1, 0, 3, 7, 30], labels=['None', 'Low', 'Moderate', 'High'])

# number of hours a person sleeps
df['SleepTime_cat'] = pd.cut(df['SleepTime'], bins=[0, 4, 7, 9, 24], labels=['Very Short', 'Short', 'Normal', 'Very Long'])

print(df.head())

  HeartDisease    BMI Smoking  ... PhysicalHealth_cat MentalHealth_cat  SleepTime_cat
0           No  16.60     Yes  ...                Low             High          Short
1           No  20.34      No  ...               None             None          Short
2           No  26.58     Yes  ...               High             High         Normal
3           No  24.21      No  ...               None             None          Short
4           No  23.71      No  ...               High             None         Normal

[5 rows x 22 columns]


### Combining Rare Categories

In [51]:
# Combine rare categories before one-hot encoding 
df['Race'] = df['Race'].replace(['American Indian/Alaskan Native', 'Other'], 'Other')

# Merge very short and short sleep categories
df['SleepTime_cat'] = df['SleepTime_cat'].replace(
    ['Very Short', 'Short'], 'Short'
)

print("Race column value counts:")
print(df['Race'].value_counts())

print("\nSleepTime_cat column value counts:")
print(df['SleepTime_cat'].value_counts())


Race column value counts:
Race
White       245212
Hispanic     27446
Black        22939
Other        16130
Asian         8068
Name: count, dtype: int64

SleepTime_cat column value counts:
SleepTime_cat
Short        194737
Normal       113643
Very Long     11415
Name: count, dtype: int64


C:\Users\Admin\AppData\Local\Temp\ipykernel_5936\2286667650.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df['SleepTime_cat'] = df['SleepTime_cat'].replace(


### Transactional Data
Output: Sparse DataFrame (each column is a specific medical feature/risk level).

In [52]:
# Select the categorical columns
categorical_cols = ['HeartDisease', 'BMI_cat', 'Smoking', 'AlcoholDrinking', 'Stroke', 'PhysicalHealth_cat', 'MentalHealth_cat', 
                    'DiffWalking', 'Sex', 'AgeCategory', 'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime_cat', 
                    'Asthma', 'KidneyDisease', 'SkinCancer']

df_categorical = df[categorical_cols]

# One-hot encode the categorical variables
df_encoded = pd.get_dummies(df_categorical)
print('\n Dataframe after performing One-Hot Encoding:')
display(df_encoded)


 Dataframe after performing One-Hot Encoding:


,HeartDisease_No,HeartDisease_Yes,BMI_cat_Underweight,BMI_cat_Normal,BMI_cat_Overweight,BMI_cat_Obese,Smoking_No,Smoking_Yes,AlcoholDrinking_No,AlcoholDrinking_Yes,Stroke_No,Stroke_Yes,PhysicalHealth_cat_None,PhysicalHealth_cat_Low,PhysicalHealth_cat_Moderate,PhysicalHealth_cat_High,MentalHealth_cat_None,MentalHealth_cat_Low,MentalHealth_cat_Moderate,MentalHealth_cat_High,DiffWalking_No,DiffWalking_Yes,Sex_Female,Sex_Male,AgeCategory_18-24,AgeCategory_25-29,AgeCategory_30-34,AgeCategory_35-39,AgeCategory_40-44,AgeCategory_45-49,AgeCategory_50-54,AgeCategory_55-59,AgeCategory_60-64,AgeCategory_65-69,AgeCategory_70-74,AgeCategory_75-79,AgeCategory_80 or older,Race_Asian,Race_Black,Race_Hispanic,Race_Other,Race_White,Diabetic_No,"Diabetic_No, borderline diabetes",Diabetic_Yes,Diabetic_Yes (during pregnancy),PhysicalActivity_No,PhysicalActivity_Yes,GenHealth_Excellent,GenHealth_Fair,GenHealth_Good,GenHealth_Poor,GenHealth_Very good,SleepTime_cat_Short,SleepTime_cat_Normal,SleepTime_cat_Very Long,Asthma_No,Asthma_Yes,KidneyDisease_No,KidneyDisease_Yes,SkinCancer_No,SkinCancer_Yes
0,True,False,True,False,False,False,False,True,True,False,True,False,False,True,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,False,False,True,True,False,False,False,True,True,False,False,True
1,True,False,False,True,False,False,True,False,True,False,False,True,True,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,True,True,False,False,True,False,True,False,True,False
2,True,False,False,False,True,False,False,True,True,False,True,False,False,False,False,True,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,True,False,False,False,False,True,False,False,True,True,False,True,False
3,True,False,False,True,False,False,True,False,True,False,True,False,True,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,False,True,False,False,False,True,False,False,True,False,False,True,False,True,False,False,True
4,True,False,False,True,False,False,True,False,True,False,True,False,False,False,False,True,True,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,True,False,True,False,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,False,True,False,False,True,False,False,True,True,False,True,False,False,False,True,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False,False,True,False,False,False,True,True,False,True,False
319791,True,False,False,False,True,False,False,True,True,False,True,False,True,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,True,True,False,False,False,True,True,False,True,False
319792,True,False,False,True,False,False,True,False,True,False,True,False,True,False,False,False,True,False,False,False,True,False,True,False,False,False,

### Data Pruning 

In [53]:
# Drop columns with very low variance

# Original number of columns
print(f"Original number of columns: {df_encoded.shape[1]}")

# Remove very low variance columns (<1%)
selector = VarianceThreshold(threshold=0.01) 
df_pruned_array = selector.fit_transform(df_encoded)

# Convert back to DataFrame
df_pruned = pd.DataFrame(df_pruned_array, columns=df_encoded.columns[selector.get_support()])

# Number of columns after pruning
print(f"Number of columns after pruning: {df_pruned.shape[1]}")

# See which columns were removed
dropped_cols = df_encoded.columns[~selector.get_support()]
print("\nDropped columns (very low variance):")
print(dropped_cols.tolist())

# See the pruned dataframe
print("\nPruned dataset sample:")
print(df_pruned.head())

Original number of columns: 62
Number of columns after pruning: 61

Dropped columns (very low variance):
['Diabetic_Yes (during pregnancy)']

Pruned dataset sample:
   HeartDisease_No  HeartDisease_Yes  ...  SkinCancer_No  SkinCancer_Yes
0             True             False  ...          False            True
1             True             False  ...           True           False
2             True             False  ...           True           False
3             True             False  ...          False            True
4             True             False  ...           True           False

[5 rows x 61 columns]


### Save preprocessed data in csv file format

In [54]:
df_preprocessed = df_pruned.copy()

# Save the preprocessed dataframe to a new CSV file
df_preprocessed.to_csv(r'C:\Users\Admin\Documents\dataset_association_mining\heart_disease_preprocessed.csv', index=False)